In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os, codecs, string, random
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle

%load_ext autoreload
%autoreload 2
%matplotlib inline

import re
from tqdm import tqdm

import warnings; warnings.simplefilter('ignore')

import spacy, nltk, gensim
import pyLDAvis.gensim

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [2]:
seed = 1
random.seed(seed)
dirnames = "data/"+os.listdir("data")[0]
dirnames += "/"+ os.listdir(dirnames)[0]+"/"

In [3]:
os.listdir(dirnames)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
from tqdm import tqdm, tqdm_notebook

In [5]:
# implement this later for clarity
def read_file():
    pass
def read_directory():
    pass
def save_data():
    pass

In [27]:
data.to_pickle("./data.pkl")
unpickled_df = pd.read_pickle("./data.pkl")
unpickled_df

,labels,file_id,newsgroups,subject,organization,date,n_lines,text
0,"[alt, atheism]",49960,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Atheist Resources,"Mantis Consultants, Cambridge. UK.","Mon, 29 Mar 1993 11:57:19 GMT",290,\nArchive-name: atheism/resources\nAlt-atheism...
0,"[alt, atheism]",51060,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Introduction to Atheism,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 12:22:45 GMT",646,\nArchive-name: atheism/introduction\nAlt-athe...
0,"[alt, atheism]",51119,[alt.atheism],Re: Gospel Dating,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:08:25 GMT",93,\nIn article <65974@mimsy.umd.edu>\nmangoe@cs....
0,"[alt, atheism]",51120,"[alt.atheism, alt.politics.usa.constitution]",Re: university violating separation of church/...,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 17:58:42 GMT",29,\ndmn@kepler.unh.edu (...until kings become ph...
0,"[alt, atheism]",51121,"[alt.atheism, soc.motss, rec.scouting]","Re: [soc.motss, et al.] ""Princeton axes matchi...",IBM Research,"Mon, 05 Apr 93 18:01:16 GMT",15,\nIn article <N4HY.93Apr5120934@harder.ccr-p.i...
0,"[alt, atheism]",51122,[alt.atheism],Re: A visit from the Jehovah's Witnesses,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:24:19 GMT",114,\nIn article <1993Apr5.091139.823@batman.bmd.t...
0,"[alt, atheism]",51123,[alt.atheism],Re: Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 93 19:05:57 GMT,11,NNTP-Posting-Host: punisher.caltech.edu\n\narr...
0,"[alt, atheism]",51124,[alt.atheism],Re: An Anecdote about Islam,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:45:34 GMT",28,\nIn article <114127@bu.edu>\njaeger@buphy.bu....
0,"[alt, atheism]",51125,[alt.atheism],Re: <Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 1993 20:43:17 GMT,54,Message-ID: <1pi8h5INNq40@gap.caltech.edu>\nRe...
0,"[alt, atheism]",51126,[alt.atheism],Re: >>>>>>Pompous ass,"California Institute of Technology, Pasadena",2 Apr 93 20:57:33 GMT,9,NNTP-Posting-Host: punisher.caltech.edu\n\nkmr...


In [28]:
os.remove("./data.pkl")

In [6]:
data = pd.DataFrame(columns = ["labels","file_id","newsgroups","subject","organization","date","n_lines","text"])

for dirname in tqdm_notebook(os.listdir(dirnames)):
    dirpath = dirnames + dirname + "/"
    
    for filename in tqdm(os.listdir(dirpath)):
        filepath = dirpath + filename
        
        with open(filepath) as current_file:
    #         should check that all files have the same number of digits in the IID :////
            labels = dirname.split(".")
            file_id = current_file.name[-5:]
        
            doc = current_file.read()
            if(re.search("Newsgroups: .*\n", doc) and re.findall("Newsgroups: .*\n", doc)[0]):
                newsgroups = re.findall("Newsgroups: .*\n", doc)[0]\
                                .split("\n")[0].split("Newsgroups: ")[-1].split(",")

            if(re.search("Subject: .*\n", doc) and re.findall("Subject: .*\n", doc)[0]):
                subject = re.findall("Subject: .*\n", doc)[0]\
                                .split("\n")[0].split("Subject: ")[-1]

            if(re.search("Organization: .*\n", doc) and re.findall("Organization: .*\n", doc)[0]):
                organization = re.findall("Organization: .*\n", doc)[0]\
                                .split("\n")[0].split("Organization: ")[-1]

            if(re.search("Date: .*\n", doc) and re.findall("Date: .*\n", doc)[0]):
                date = re.findall("Date: .*\n", doc)[0]\
                                .split("\n")[0].split("Date: ")[-1]

            if(re.search("Lines: .*\n", doc) and re.findall("Lines: .*\n", doc)[0]):
                n_lines = re.findall("Lines: .*\n", doc)[0]\
                                .split("\n")[0].split("Lines: ")[-1]
            
    #         make sure it exists
#         text = "".join(re.split(".*@.*","".join(re.split("\nLines: [0123456789]+\n\n", doc)[1:])))
# were saying that there always exists these stuff,,, basically we'll drop anything not having these
# since we're taking the first split to be the preamble, if there is no first preamble split but one later, it will
# remove part of the text, but we will say that this doesnt happen too often
#  also basically we're saying that "Lines" is a stopword! since we're splitting on it :/ 
# we could split and add it back but it makes sense to keep it as a stopword in most cases except geometry / else.... 
# but at the same time, looking at the list of stopwords we are going to use later, there are words such as 
# "keep" which has th same issue! so we're fine.
            text = "".join(re.split("\nLines: [0123456789]+\n", doc)[1:])
            data = data.append(pd.DataFrame(np.array([[labels, file_id, newsgroups, subject, organization, date, n_lines, text]]),\
                                        columns=["labels","file_id","newsgroups","subject","organization","date","n_lines","text"]))

data_backup = data.copy(deep = True)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 62.09it/s]


In [7]:
data.head(5)

,labels,file_id,newsgroups,subject,organization,date,n_lines,text
0,"[alt, atheism]",49960,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Atheist Resources,"Mantis Consultants, Cambridge. UK.","Mon, 29 Mar 1993 11:57:19 GMT",290,\nArchive-name: atheism/resources\nAlt-atheism...
0,"[alt, atheism]",51060,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Introduction to Atheism,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 12:22:45 GMT",646,\nArchive-name: atheism/introduction\nAlt-athe...
0,"[alt, atheism]",51119,[alt.atheism],Re: Gospel Dating,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:08:25 GMT",93,\nIn article <65974@mimsy.umd.edu>\nmangoe@cs....
0,"[alt, atheism]",51120,"[alt.atheism, alt.politics.usa.constitution]",Re: university violating separation of church/...,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 17:58:42 GMT",29,\ndmn@kepler.unh.edu (...until kings become ph...
0,"[alt, atheism]",51121,"[alt.atheism, soc.motss, rec.scouting]","Re: [soc.motss, et al.] ""Princeton axes matchi...",IBM Research,"Mon, 05 Apr 93 18:01:16 GMT",15,\nIn article <N4HY.93Apr5120934@harder.ccr-p.i...


In [8]:
data.shape

(19997, 8)

In [62]:
temp = data.copy()
at= temp.iloc[0:10]
at

,labels,file_id,newsgroups,subject,organization,date,n_lines,text
0,"[alt, atheism]",49960,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Atheist Resources,"Mantis Consultants, Cambridge. UK.","Mon, 29 Mar 1993 11:57:19 GMT",290,\nArchive-name: atheism/resources\nAlt-atheism...
0,"[alt, atheism]",51060,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Introduction to Atheism,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 12:22:45 GMT",646,\nArchive-name: atheism/introduction\nAlt-athe...
0,"[alt, atheism]",51119,[alt.atheism],Re: Gospel Dating,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:08:25 GMT",93,\nIn article <65974@mimsy.umd.edu>\nmangoe@cs....
0,"[alt, atheism]",51120,"[alt.atheism, alt.politics.usa.constitution]",Re: university violating separation of church/...,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 17:58:42 GMT",29,\ndmn@kepler.unh.edu (...until kings become ph...
0,"[alt, atheism]",51121,"[alt.atheism, soc.motss, rec.scouting]","Re: [soc.motss, et al.] ""Princeton axes matchi...",IBM Research,"Mon, 05 Apr 93 18:01:16 GMT",15,\nIn article <N4HY.93Apr5120934@harder.ccr-p.i...
0,"[alt, atheism]",51122,[alt.atheism],Re: A visit from the Jehovah's Witnesses,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:24:19 GMT",114,\nIn article <1993Apr5.091139.823@batman.bmd.t...
0,"[alt, atheism]",51123,[alt.atheism],Re: Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 93 19:05:57 GMT,11,NNTP-Posting-Host: punisher.caltech.edu\n\narr...
0,"[alt, atheism]",51124,[alt.atheism],Re: An Anecdote about Islam,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:45:34 GMT",28,\nIn article <114127@bu.edu>\njaeger@buphy.bu....
0,"[alt, atheism]",51125,[alt.atheism],Re: <Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 1993 20:43:17 GMT,54,Message-ID: <1pi8h5INNq40@gap.caltech.edu>\nRe...
0,"[alt, atheism]",51126,[alt.atheism],Re: >>>>>>Pompous ass,"California Institute of Technology, Pasadena",2 Apr 93 20:57:33 GMT,9,NNTP-Posting-Host: punisher.caltech.edu\n\nkmr...


In [10]:
from collections import Counter
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS

In [11]:
nlp = spacy.load("en_core_web_sm")

In [57]:
EXCLUDE_CHARS = list(set(punctuation).union(set('’')))

In [59]:
[EXCLUDE_CHARS.append("\\"+ str(i)) for i in range(10)]

[None, None, None, None, None, None, None, None, None, None]

In [60]:
EXCLUDE_CHARS

['*',
 '&',
 '%',
 ']',
 '$',
 ')',
 '"',
 '(',
 '=',
 '{',
 '/',
 ';',
 '?',
 ',',
 '_',
 '<',
 '`',
 '~',
 '[',
 '\\',
 '>',
 ':',
 '.',
 '|',
 '^',
 '+',
 '@',
 '}',
 '’',
 "'",
 '-',
 '#',
 '!',
 '\\0',
 '\\1',
 '\\2',
 '\\3',
 '\\4',
 '\\5',
 '\\6',
 '\\7',
 '\\8',
 '\\9']

In [61]:
# remove stopwords and then lemmatize
def lemmatizer(doc):
    words = [token.lemma_ for token in nlp(doc) if token.is_stop != True \
             and token.is_punct != True \
             and token.is_space != True \
             and re.search("\\"+"|\\".join(EXCLUDE_CHARS), token.lemma_) is None]
    return words

In [66]:
clutch = "This is a sentence that is made to be 12h4 j@j"

In [67]:
w = lemmatizer(clutch)
w

['this', 'sentence', '12h4']

In [53]:
import time

In [63]:
t1 = time.time()
at["text"] = at.text.apply(lambda x: lemmatizer(x))
t2 = time.time()
print(t2-t1)

10.944839239120483


In [64]:
bag_of_words = Counter(at.text.explode())

In [65]:
bag_of_words

Counter({'archive': 6,
         'atheism': 57,
         'resource': 2,
         'alt': 2,
         'last': 2,
         'modify': 2,
         '11': 1,
         'december': 1,
         '1992': 2,
         'version': 8,
         'atheist': 100,
         'resources': 2,
         'addresses': 1,
         'organizations': 1,
         'usa': 7,
         'freedom': 3,
         'from': 3,
         'religion': 50,
         'foundation': 3,
         'darwin': 4,
         'fish': 6,
         'bumper': 1,
         'sticker': 1,
         'assorted': 2,
         'paraphernalia': 1,
         'available': 2,
         'us': 3,
         'write': 19,
         'ffrf': 1,
         'box': 4,
         '750': 1,
         'madison': 1,
         'wi': 1,
         '53701': 1,
         'telephone': 4,
         '608': 1,
         '256': 1,
         '8900': 1,
         'evolution': 3,
         'design': 2,
         'sell': 2,
         'be': 52,
         'symbol': 1,
         'like': 11,
         'one': 7,
         '

In [42]:
at

,labels,file_id,newsgroups,subject,organization,date,n_lines,text
0,"[alt, atheism]",49960,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Atheist Resources,"Mantis Consultants, Cambridge. UK.","Mon, 29 Mar 1993 11:57:19 GMT",290,"[archive, atheism, resource, alt, atheism, arc..."
0,"[alt, atheism]",51060,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Introduction to Atheism,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 12:22:45 GMT",646,"[archive, atheism, introduction, alt, atheism,..."
0,"[alt, atheism]",51119,[alt.atheism],Re: Gospel Dating,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:08:25 GMT",93,"[in, article, charley, wingate, write, well, j..."
0,"[alt, atheism]",51120,"[alt.atheism, alt.politics.usa.constitution]",Re: university violating separation of church/...,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 17:58:42 GMT",29,"[king, philosopher, philosopher, king, write, ..."
0,"[alt, atheism]",51121,"[alt.atheism, soc.motss, rec.scouting]","Re: [soc.motss, et al.] ""Princeton axes matchi...",IBM Research,"Mon, 05 Apr 93 18:01:16 GMT",15,"[in, article, bob, mcgwier, write, 1, however,..."
0,"[alt, atheism]",51122,[alt.atheism],Re: A visit from the Jehovah's Witnesses,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:24:19 GMT",114,"[in, article, write, do, not, lucifer, create,..."
0,"[alt, atheism]",51123,[alt.atheism],Re: Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 93 19:05:57 GMT,11,"[nntp, posting, host, ken, arromdee, write, th..."
0,"[alt, atheism]",51124,[alt.atheism],Re: An Anecdote about Islam,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:45:34 GMT",28,"[in, article, gregg, jaeger, write, when, vict..."
0,"[alt, atheism]",51125,[alt.atheism],Re: <Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 1993 20:43:17 GMT,54,"[message, id, reference, nntp, posting, host, ..."
0,"[alt, atheism]",51126,[alt.atheism],Re: >>>>>>Pompous ass,"California Institute of Technology, Pasadena",2 Apr 93 20:57:33 GMT,9,"[nntp, posting, host, keith, ryan, write, then..."


In [20]:
print(nlp.Defaults.stop_words)

{'just', 'until', 'whereupon', 'see', 'anywhere', 'whence', 'after', 'amount', 'i', 'your', 'full', 'upon', 'himself', 'almost', 'hereby', 'five', 'moreover', 'third', 'already', 'otherwise', 'those', 'had', 'indeed', 'nobody', 'under', 'during', 'through', 'its', 'mine', 'him', 'per', 'my', 'make', 'he', 'whom', 'keep', 'few', 'everywhere', 'further', 'wherein', 'whoever', 'whole', 'across', 'by', 'can', 'myself', 'for', 'our', 'due', 'elsewhere', 'below', 'meanwhile', 'his', 'perhaps', 'thereby', 'and', 'where', 'without', 'it', 'too', 'again', 'besides', 'noone', 'eleven', 'more', 're', 'still', 'get', 'yet', 'yours', 'six', 'whose', 'am', 'up', 'anyway', 'beyond', 'often', 'rather', 'how', 'than', 'while', 'two', 'do', 'an', 'say', 'are', 'sometimes', 'because', 'beside', 'once', 'never', 'onto', 'over', 'ten', 'someone', 'whereby', 'me', 'else', 'so', 'should', 'although', 'unless', 'to', 'latterly', 'this', 'eight', 'however', 'whereas', 'mostly', 'around', 'before', 'whither', '

In [ ]:
# should also have the dirname where it comes form
# should check that we have roufhly the same amount of datapoints in each class

In [21]:
import nltk

In [22]:
from nltk.corpus import stopwords


In [23]:
stopWords = set(stopwords.words('english'))
# join with other set and .unique()
# also should remove numbers that are within words

In [24]:
stopWords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
data.text = data.text.apply(lambda x: lemmatizer(x))

In [ ]:
bag_of_words = Counter(data.text.explode())

In [ ]:
train_test_split()

In [29]:
def one_hot_labels():
    pass